In [15]:
from collections import deque
from dotenv import load_dotenv
from label_studio_sdk.client import LabelStudio
from glob import glob
from pathlib import Path
from sqlite3 import connect
from typing import Tuple
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sklearn.linear_model
from scipy.ndimage import map_coordinates
from tqdm import tqdm_notebook
from sklearn.linear_model import LinearRegression
import plotly.graph_objs as go
from typing import Any



In [16]:
%matplotlib widget

In [17]:
load_dotenv()

True

In [18]:
LABEL_STUDIO_URL = "https://labeler.e4e.ucsd.edu"
LABEL_STUDIO_API_KEY = os.environ["LABEL_STUDIO_API_KEY"]
LABEL_STUDIO_PROJECT_ID = 46

LABEL_STUDIO_URL, LABEL_STUDIO_API_KEY, LABEL_STUDIO_PROJECT_ID

('https://labeler.e4e.ucsd.edu',
 '81c03c9417007e80fad61214587317d08d306a3d',
 46)

In [19]:
ipad_camera_intrinsics = np.array([[1604.2147, 0.0, 956.5816],
                                   [0.0, 1604.2147, 717.7617],
                                   [0.0, 0.0, 1.0]])
iphone_camera_intrinsics = np.array([[1375.0719, 0.0, 968.6433],
                                     [0.0, 1375.0719, 723.04926],
                                     [0.0, 0.0, 1.0]])

In [20]:
label_studio = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=LABEL_STUDIO_API_KEY)

label_studio

In [21]:
# path containing 2025.06.27.FishSense.SanDiego
PATH_TO_DATA = Path(os.environ["PATH_TO_DATA"])

In [22]:
# rgbs = [Path(g).absolute().resolve() for g in glob("../data/2025.06.27.FishSense.SanDiego/**/**/FSM/rgb_*.jpg", recursive=True)]
rgbs = [(PATH_TO_DATA / Path(g)).resolve() for g in glob("2025.06.27.FishSense.SanDiego/**/**/FSM/rgb_*.jpg", root_dir=PATH_TO_DATA, recursive=True)]
timestamp_to_rgb_map = {int(f.stem[4:]):f for f in rgbs}

rgbs

[PosixPath('/Volumes/fishsense_data/CCFRP/2025.06.27.FishSense.SanDiego/ED-00/iPad/FSM/rgb_1751059596.jpg'),
 PosixPath('/Volumes/fishsense_data/CCFRP/2025.06.27.FishSense.SanDiego/ED-00/iPad/FSM/rgb_1751059200.jpg'),
 PosixPath('/Volumes/fishsense_data/CCFRP/2025.06.27.FishSense.SanDiego/ED-00/iPad/FSM/rgb_1751059821.jpg'),
 PosixPath('/Volumes/fishsense_data/CCFRP/2025.06.27.FishSense.SanDiego/ED-00/iPad/FSM/rgb_1751059518.jpg'),
 PosixPath('/Volumes/fishsense_data/CCFRP/2025.06.27.FishSense.SanDiego/ED-00/iPad/FSM/rgb_1751047797.jpg'),
 PosixPath('/Volumes/fishsense_data/CCFRP/2025.06.27.FishSense.SanDiego/ED-00/iPad/FSM/rgb_1751048739.jpg'),
 PosixPath('/Volumes/fishsense_data/CCFRP/2025.06.27.FishSense.SanDiego/ED-00/iPad/FSM/rgb_1751061242.jpg'),
 PosixPath('/Volumes/fishsense_data/CCFRP/2025.06.27.FishSense.SanDiego/ED-00/iPad/FSM/rgb_1751047800.jpg'),
 PosixPath('/Volumes/fishsense_data/CCFRP/2025.06.27.FishSense.SanDiego/ED-00/iPad/FSM/rgb_1751047788.jpg'),
 PosixPath('/Volume

In [23]:
def get_depth_map_and_confidence_map(timestamp: int, database_path: Path) -> Tuple[np.ndarray[float], np.ndarray[int]]:
    with connect(database_path) as connection:
        cursor = connection.cursor()

        query = """
        SELECT depth_bytes, depth_width, depth_height, confidence_bytes, confidence_width, confidence_height FROM photos
        WHERE utc_unix_timestamp = ?"""

        cursor.execute(query, (timestamp,))
        row = cursor.fetchone()

        depth_width = row[1]
        depth_height = row[2]
        depth_map = np.frombuffer(row[0], dtype=np.float32).reshape((depth_height, depth_width))

        confidence_width = row[4]
        confidence_height = row[5]
        confidence_map = np.frombuffer(row[3], dtype=np.int8).reshape((confidence_height, confidence_width))

    return depth_map, confidence_map

In [24]:
def get_neighbors(depth_map: np.ndarray[float], node: np.ndarray[int], epsilon=0.005):
    height, width = depth_map.shape
    max_pos = np.array([width, height])

    up = node + np.array([0, -1])
    down = node + np.array([0, 1])
    left = node + np.array([-1, 0])
    right = node + np.array([1, 0])

    if np.all(up < max_pos) and np.all(up >= 0) and np.all(np.abs(depth_map[node[1], node[0]] - depth_map[up[1], up[0]]) <= epsilon):
        yield up
    if np.all(down < max_pos) and np.all(down >= 0) and np.all(np.abs(depth_map[node[1], node[0]] - depth_map[down[1], down[0]]) <= epsilon):
        yield down
    if np.all(left < max_pos) and np.all(left >= 0) and np.all(np.abs(depth_map[node[1], node[0]] - depth_map[left[1], left[0]]) <= epsilon):
        yield left
    if np.all(right < max_pos) and np.all(right >= 0) and np.all(np.abs(depth_map[node[1], node[0]] - depth_map[right[1], right[0]]) <= epsilon):
        yield right

def correct_labels(depth_map: np.ndarray[float], depth_annotations: np.ndarray[float]):
    x2 = depth_annotations[:, 0].max()
    x1 = depth_annotations[:, 0].min()

    y2 = depth_annotations[:, 1].max()
    y1 = depth_annotations[:, 1].min()

    x_middle = x1 + (x2 - x1) / 2
    y_middle = y1 + (y2 - y1) / 2

    start_node = np.round(np.array([x_middle, y_middle])).astype(int)

    visited = set()
    current_component = []
    queue = deque([start_node])
    visited.add(tuple(start_node))

    while queue:
        current_node = queue.popleft()
        current_component.append(current_node)

        for neighbor in get_neighbors(depth_map, current_node):
            if tuple(neighbor) not in visited:
                visited.add(tuple(neighbor))
                queue.append(neighbor)

    current_component = np.array(current_component)

    arg_0 = np.argmin(np.sum((depth_annotations[0, :] - np.array(current_component))**2, axis=1))
    depth_annotations[0, :] = current_component[arg_0, :]

    arg_1 = np.argmin(np.sum((depth_annotations[1, :] - np.array(current_component))**2, axis=1))
    depth_annotations[1, :] = current_component[arg_1, :]

    return depth_annotations

In [25]:
def plane_ransac(depth_array):
    """
    Perform RANSAC to fit a plane to the depth data.
    Returns the coefficients of the plane equation Ax + By + C = z.
    """

    # # Fit a plane using RANSAC
    from sklearn.linear_model import RANSACRegressor, LinearRegression
    
    
    XY = depth_array[0:2,:].T
    Z = depth_array[2,:]
    
    model = RANSACRegressor(LinearRegression(), min_samples=3)
    model.fit(XY, Z)
    

    return model.estimator_.coef_[0], model.estimator_.coef_[1], model.estimator_.intercept_

In [37]:
def compute_fish_length(task: Any, timestamp: int, rgb_path: Path, database_path: Path) -> Tuple[float, int, np.ndarray[np.uint8], np.ndarray[float], np.ndarray[np.uint8], np.ndarray[float], np.ndarray[float]]:
    rgb_image = cv2.imread(str(rgb_path))   
    depth_map, confidence_map = get_depth_map_and_confidence_map(timestamp, database_path)

    snout_results = [r for a in task.annotations for r in a["result"] if any(k == "Snout" for k in r["value"]["keypointlabels"])]
    fork_results = [r for a in task.annotations for r in a["result"] if any(k == "Fork" for k in r["value"]["keypointlabels"])]
        
    snout = snout_results[0]
    fork = fork_results[0]

    snout = [snout["value"]["x"] / 100, snout["value"]["y"] / 100]
    fork = [fork["value"]["x"] / 100, fork["value"]["y"] / 100]

    annotations = np.array([snout, fork])

    rgb_height, rgb_width, _ = rgb_image.shape
    rgb_shape = np.array([rgb_width, rgb_height])
    rgb_annotations = annotations * rgb_shape

    depth_height, depth_width = depth_map.shape
    depth_map_shape = np.array([depth_width, depth_height])
    depth_annotations = correct_labels(depth_map, annotations * depth_map_shape)

    camera_intrinsics = ipad_camera_intrinsics if "iPad" in str(rgb_path) else iphone_camera_intrinsics

    depths = depth_map[depth_annotations[:, 1].astype(int), depth_annotations[:, 0].astype(int)]
    confidences = confidence_map[depth_annotations[:, 1].astype(int), depth_annotations[:, 0].astype(int)]

    points3d = np.linalg.inv(camera_intrinsics) @ np.array([rgb_annotations[:, 0], rgb_annotations[:, 1], np.ones_like(rgb_annotations[:, 0])])
    for idx, depth_pixel in enumerate(depths):
        points3d[idx, :] *= depth_pixel

    fish_length = np.linalg.norm(points3d[:, 0] - points3d[:, 1])
    confidence = np.min(confidences)
    
    ransac_mesh = np.meshgrid(np.arange(rgb_width), np.arange(rgb_height))
    
    ransac_points = np.linalg.inv(camera_intrinsics) @ np.array([ransac_mesh[0].ravel(), ransac_mesh[1].ravel(), np.ones(rgb_width*rgb_height)])
    
    rgb_points = np.stack((ransac_mesh[1].ravel(), ransac_mesh[0].ravel()), axis=1)
    
    percent_points = (rgb_points * 1.0) / ((np.array([rgb_height, rgb_width]) - 1) * 1.0)
    
    depth_points = np.floor(percent_points * (np.array([depth_height, depth_width]) - 1)).astype(int)
    depths = depth_map[depth_points[:, 0], depth_points[:, 1]]
    
    ransac_points = np.repeat(depths[:, None], 3, axis=1).T * ransac_points

    norm_vec = plane_ransac(ransac_points)
    
    angle = np.degrees(np.arccos(np.dot(norm_vec, np.array([0, 0, 1]))/(np.linalg.norm(norm_vec))))
    x_angle = np.degrees(np.arccos(np.dot(norm_vec * np.array([1, 0, 1]), np.array([0, 0, 1]))/(np.linalg.norm(norm_vec* np.array([1, 0, 1])))))
    y_angle = np.degrees(np.arccos(np.dot(norm_vec * np.array([0, 1, 1]), np.array([0, 0, 1]))/(np.linalg.norm(norm_vec* np.array([0, 1, 1])))))
    
    # return norm_vec, ransac_points, rgb_width, rgb_height, rgb_points

    return fish_length, confidence, rgb_image, depth_map, confidence_map, rgb_annotations, depth_annotations, angle, x_angle, y_angle

In [38]:
fish_length_data = []

for task in label_studio.tasks.list(project=LABEL_STUDIO_PROJECT_ID):
    timestamp = int(Path(task.storage_filename).stem[4:])
    if timestamp not in timestamp_to_rgb_map:
        continue

    if not any(a["result"] for a in task.annotations):
        continue

    rgb_path = timestamp_to_rgb_map[timestamp]
    database_path = rgb_path.parent / "database.sqlite"

    # if timestamp == 1751410897:
    #     break

    fish_length, confidence, rgb_image, depth_map, confidence_map, rgb_annotations, depth_annotations, angle, x_angle, y_angle = compute_fish_length(task, timestamp, rgb_path, database_path)
    
    # norm_vec, ransac_points, rgb_height, rgb_width, rgb_points = compute_fish_length(task, timestamp, rgb_path, database_path)
        
    parts = rgb_path.parts
    ed_value = parts[-4]
    device = parts[-3]

    fish_length_data.append({
        'timestamp': timestamp,
        'ED': ed_value,
        'device': device,
        'rgb_path': parts[-4] +"/" + parts[-3] + "/" + parts[-2] + "/" + parts[-1],
        'fish_length_mm': fish_length * 1000,
        'confidence': confidence,
        'angle': angle,
        'x_angle': x_angle,
        'y_angle': y_angle
        })     
    print(f"Timestamp {timestamp} ({ed_value}, {device}): Fish length = {fish_length * 1000:.2f} mm, Angle = {angle} degrees")
    print(x_angle, y_angle)

Timestamp 1751040799 (ED-00, iPad): Fish length = 348.72 mm, Angle = 45.6914380060284 degrees
24.34259058787435 42.58676693625418
Timestamp 1751048739 (ED-00, iPad): Fish length = 292.29 mm, Angle = 32.72192340029998 degrees
3.3217470760781493 32.61526997067945
Timestamp 1751048791 (ED-00, iPad): Fish length = 252.56 mm, Angle = 7.934034647781396 degrees
7.737568827533966 1.776216103685088
Timestamp 1751048875 (ED-00, iPad): Fish length = 757.63 mm, Angle = 6.305597097995638 degrees
5.780041472864323 2.5374260679712
Timestamp 1751050086 (ED-00, iPad): Fish length = 375.79 mm, Angle = 28.799414883866593 degrees
0.8593001155926194 28.790409975185735
Timestamp 1751050103 (ED-00, iPad): Fish length = 384.97 mm, Angle = 20.049658322600536 degrees
19.578942146378605 4.6755773423689755
Timestamp 1751057767 (ED-00, iPad): Fish length = 534.08 mm, Angle = 16.380734873486066 degrees
11.673661158462885 11.809885614021033
Timestamp 1751057870 (ED-00, iPad): Fish length = 320.91 mm, Angle = 15.7823

In [40]:
import pandas as pd
fish_length_df = pd.DataFrame(fish_length_data)
fish_length_df.to_csv("fish_length_prediction.csv")